In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

!pip install keras4torch
!pip install einops

os.chdir('/content/drive/MyDrive/ECML2/Stab/5805_s')

import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyClassLoss,CatMap,Num_Cat,MyRegreLoss
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

Mounted at /content/drive


In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Volvo')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Train=pd.read_csv('train_gen1.csv')
Variants=pd.read_csv('variants.csv')
Out=pd.read_csv('public_X_test.csv')
Variants.set_index('ChassisId_encoded',inplace=True)

In [ ]:
os.chdir('/content/drive/MyDrive/VolvoFinal')


In [ ]:
F= []
for c  in Out.columns:
  if c[:2]=='af' or c[:2]=='f_':
    F.append(c)

In [ ]:
Out.loc[Out.gen=='gen1',F] = (Out.loc[Out.gen=='gen1',F]- Train[F].quantile(0.005))
Out.loc[Out.gen=='gen2',F] = ( Out.loc[Out.gen=='gen2',F]- Out.loc[Out.gen=='gen2',F].quantile(0.005) )
Train[F] = (Train[F]- Train[F].quantile(0.005))


In [ ]:
Train['Tss']  = Train.Timesteps-Train.ChassisId_encoded.map(  Train.groupby('ChassisId_encoded').first().Timesteps)
Out['Tss']    = Out.Timesteps-Out.ChassisId_encoded.map(  Out.groupby('ChassisId_encoded').first().Timesteps)


<ipython-input-7-b045eacdaa2e>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train['Tss']  = Train.Timesteps-Train.ChassisId_encoded.map(  Train.groupby('ChassisId_encoded').first().Timesteps)


In [ ]:
  #A = Train.drop( ['Tss',	'gen'	,'Timesteps'],axis=1)
  #if train:

  #  A = Train.drop( ['Tss','risk_level',	'gen'	,'Timesteps'],axis=1)
  #A = A.diff()
  #A.loc[A.ChassisId_encoded!=0]=0
  #A['ChassisId_encoded']=Train.ChassisId_encoded

  #Amean  =  A.groupby('ChassisId_encoded').mean()
  #Amean=Amean.loc[Train.ChassisId_encoded]
  #Amean.index=Train.index

In [ ]:
def BuildXY(Train,Variants,train=True):
  #droplist = ['Timesteps'	,'Tss','ChassisId_encoded',	'gen'	]
  droplist = ['Tss','ChassisId_encoded',	'gen'	,'Timesteps']
  if train:
    droplist=droplist+['risk_level']
  TrainV=Variants.loc[Train.ChassisId_encoded]
  TrainV.index=Train.index
  X=Train.drop(droplist,axis=1)#.drop('Timesteps',axis=1)
  A = Train.drop( ['Tss',	'gen'	,'Timesteps'],axis=1)
  if train:

    A = Train.drop( ['Tss','risk_level',	'gen'	,'Timesteps'],axis=1)
  A = A.diff()
  A.loc[A.ChassisId_encoded!=0]=0
  A['ChassisId_encoded']=Train.ChassisId_encoded

  Amean  =  A.groupby('ChassisId_encoded').mean()
  Amean=Amean.loc[Train.ChassisId_encoded]
  Amean.index=Train.index

  X=pd.concat([X,TrainV*np.array( [0,0]+[0,0,0,0,0]+[0,0,0,0,0])],axis=1)

  X.columns=list(range(len(X.columns)))

  Y=None
  if train:
    L=Train.groupby('ChassisId_encoded').last()
    Train['Y']=60
    Train['sieries_size']= Train['ChassisId_encoded'].map(L.Timesteps)
    Train['sieries_end']= Train['ChassisId_encoded'].map(L.risk_level)
    Train.loc[Train.sieries_end=='High','Y']=1 + Train.loc[Train.sieries_end=='High','sieries_size'] - Train.loc[Train.sieries_end=='High','Timesteps']
    Train.loc[Train.sieries_end=='Low','Y']=Train.loc[Train.sieries_end=='Low','Y'] - Train.loc[Train.sieries_end=='Low','Timesteps']*1
    Y=Train.Y


  return X,Y


In [ ]:
def sample_pairs(data, N):
    # Check if DataFrame and columns exist
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    if 'ChassisId_encoded' not in data.columns or 'Timesteps' not in data.columns:
        raise ValueError("DataFrame must contain 'ChassisId_encoded' and 'Timesteps' columns")

    # Store indices and labels
    indices = []
    labels = []

    # Group data by 'ChassisId_encoded'
    grouped = data.groupby('ChassisId_encoded')

    # Create a list to hold possible pairs from each group
    possible_pairs = []
    for _, group in grouped:
        if len(group) >= 2:
            group_indices = group.index.tolist()
            # Generate all possible pairs in the group
            for i in range(len(group_indices)):
                for j in range(i + 1, len(group_indices)):
                    possible_pairs.append((group_indices[i], group_indices[j]))

    # Shuffle the list of possible pairs to randomize selection
    np.random.shuffle(possible_pairs)

    # Select up to N pairs from the shuffled list
    for idx1, idx2 in possible_pairs[:N]:
        # Calculate the label
        label = data.at[idx2, 'Timesteps'] - data.at[idx1, 'Timesteps']
        indices.append([idx1, idx2])
        labels.append(label)
        if len(labels) >= N:
            break

    # Convert results to the requested format
    X = np.array(indices)
    Y = np.array(labels)

    return X, Y

In [ ]:
import random

def myP(y):
  return np.where(y>2, 0.5, 1 )

def split_list(L, n, p):
    # Validate that the sum of percentages is approximately 1
    if not (0.99 <= sum(p) <= 1.01):
        raise ValueError("Sum of percentages must be approximately 1")

    # Shuffle the list to randomize the order of elements
    random.shuffle(L)

    # Calculate the sizes of each sublist
    total_length = len(L)
    sizes = [int(total_length * percentage) for percentage in p]

    # Adjust the last element size to match the total length
    sizes[-1] = total_length - sum(sizes[:-1])

    # Split the list into sublists
    result = []
    current_index = 0
    for size in sizes:
        result.append(L[current_index:current_index + size])
        current_index += size

    return result


In [ ]:
X,Y    = BuildXY(Train,Variants,train=True)
Xout,_ = BuildXY(Out,Variants,train=False)

<ipython-input-9-5844df83039a>:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train['Y']=60
<ipython-input-9-5844df83039a>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train['sieries_size']= Train['ChassisId_encoded'].map(L.Timesteps)
<ipython-input-9-5844df83039a>:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newf

In [ ]:

def datasplit(X_,Train_,size=500000,PP=[0.8,0.2]):
  X=deepcopy(X_)
  Train=deepcopy(Train_)
  train_l,test_l=split_list(Train.ChassisId_encoded.unique(),2,PP)
  Itrain=X[Train.ChassisId_encoded.isin(train_l)].index
  Itest=X[Train.ChassisId_encoded.isin(test_l)].index
  X_test,y_test=X.loc[Itest].reset_index().drop('index',axis=1),Y.loc[Itest].reset_index().drop('index',axis=1)
  X_train,y_train=X.loc[Itrain].reset_index().drop('index',axis=1),Y.loc[Itrain].reset_index().drop('index',axis=1)
  prob=myP(y_train)[:,0]


  I=X_train.sample(size, weights=prob,replace=True).index
  X_train = X_train.loc[I]
  y_train = y_train.loc[I]
  y_train=y_train.Y
  return  X_train ,y_train ,X_test,y_test,Itest


In [ ]:
import  torch.nn.functional as     F
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


def BuildModel():
  Or_model = MainModel(
      categories        = (),
      num_continuous    = 316,
      dim               = 128  ,
      dim_out           = 2,
      depth             = 2,
      heads             = 8,
      attn_dropout      = 0.2 ,
      ff_dropout        = 0.2,
      U                 = 2,
      cases             = 16

  )
  no_model = Num_Cat(Or_model,num_number=316,classes=2)
  model    = keras4torch.Model(no_model,).build([316])
  return Or_model,no_model,model


def TrainModel(checkpoint,X_train,y_train,X_test,y_test):



  Or_model,no_model,model =BuildModel()
  #Warm UP
  optimizer=torch.optim.AdamW(model.parameters(),lr=0.00001,weight_decay=0.0001,)
  sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.001, total_iters=10,  verbose=False)
  model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
  callbacks=[LRScheduler(sch)]
  no_model.reset_Sample_size(8)

  model.fit(X_train.fillna(0).values, y_train[0].values,
                        epochs=2, batch_size=256,
                        validation_data=(X_test.fillna(0).values,y_test[0].values),
                        verbose=1,validation_batch_size=32,
                        callbacks=callbacks)
  no_model.reset_Sample_size(8)

  #Main Train
  optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
  model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
  scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5,min_lr=0.00001))
  callbacks=[ModelCheckpoint(checkpoint,monitor='val_cross_entropy',mode='min'),scheduler]
  model.fit(X_train.fillna(0).values, y_train[0].values,
          epochs=40, batch_size=256,
          validation_data=(X_test.fillna(0).values,y_test[0].values),
          verbose=1,validation_batch_size=32,
          callbacks=callbacks)


  optimizer.zero_grad()  # Zero gradients at the start of your 'batch'
  del no_model
  del Or_model
  del model
  torch.cuda.empty_cache()  # This wi
  gc.collect()

 # model.load_weights(checkpoint)
  return None

In [ ]:
  for i in range(5):
      X_train ,y_train ,X_test,y_test,Itest= datasplit(X,Train,size=100000,PP=[0.75,0.25])
      X_train=X_train[(y_train<3) | ( (y_train>7) &  (y_train<13))  ].fillna(0)
      y_train=y_train[(y_train<3) | ( (y_train>7) &  (y_train<13))  ]
      Ytruth = y_test*1.0
      Xtruth =  X_test.fillna(0)*1.0
      X_test=X_test[(y_test.Y<3) | ( (y_test.Y>7) &  (y_test.Y<13))  ].fillna(0)
      y_test=y_test[(y_test.Y<3) | ( (y_test.Y>7) &  (y_test.Y<13))  ]

      Ytruth = y_test*1.0
      y_test=pd.DataFrame(np.where(y_test>3,1,0).astype(int)).fillna(0)
      y_train=pd.DataFrame(np.where(y_train>3,1,0).astype(int)).fillna(0)

      model1=TrainModel('ShortModel{}'.format(i),X_train,y_train,X_test,y_test)



/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4978 samples, validate on 1387 samples:
Epoch 1/2
20/20 - 15s - loss: 0.7732 - acc: 0.4885 - cross_entropy: 0.7794 - val_loss: 0.7911 - val_acc: 0.3562 - val_cross_entropy: 0.7974 - lr: 1e-08
Epoch 2/2
20/20 - 10s - loss: 0.7665 - acc: 0.4807 - cross_entropy: 0.7726 - val_loss: 0.7645 - val_acc: 0.3713 - val_cross_entropy: 0.7706 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4978 samples, validate on 1387 samples:
Epoch 1/40
20/20 - 10s - loss: 0.8687 - acc: 0.5034 - cross_entropy: 0.8757 - val_loss: 0.7589 - val_acc: 0.3461 - val_cross_entropy: 0.7648 - lr: 1e-03
Epoch 2/40
20/20 - 10s - loss: 0.6909 - acc: 0.5173 - cross_entropy: 0.6960 - val_loss: 0.6893 - val_acc: 0.4917 - va

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4962 samples, validate on 1403 samples:
Epoch 1/2
20/20 - 10s - loss: 0.7777 - acc: 0.5000 - cross_entropy: 0.7839 - val_loss: 0.6362 - val_acc: 0.6693 - val_cross_entropy: 0.6409 - lr: 1e-08
Epoch 2/2
20/20 - 10s - loss: 0.7695 - acc: 0.4929 - cross_entropy: 0.7756 - val_loss: 0.6410 - val_acc: 0.6657 - val_cross_entropy: 0.6458 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4962 samples, validate on 1403 samples:
Epoch 1/40
20/20 - 10s - loss: 0.8602 - acc: 0.5169 - cross_entropy: 0.8672 - val_loss: 0.6438 - val_acc: 0.6728 - val_cross_entropy: 0.6485 - lr: 1e-03
Epoch 2/40
20/20 - 10s - loss: 0.6923 - acc: 0.5195 - cross_entropy: 0.6975 - val_loss: 0.6682 - val_acc: 0.6237 - va

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4898 samples, validate on 1433 samples:
Epoch 1/2
20/20 - 10s - loss: 0.7890 - acc: 0.4902 - cross_entropy: 0.7953 - val_loss: 0.8518 - val_acc: 0.3287 - val_cross_entropy: 0.8587 - lr: 1e-08
Epoch 2/2
20/20 - 10s - loss: 0.7760 - acc: 0.4824 - cross_entropy: 0.7821 - val_loss: 0.8083 - val_acc: 0.3371 - val_cross_entropy: 0.8148 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4898 samples, validate on 1433 samples:
Epoch 1/40
20/20 - 10s - loss: 0.8438 - acc: 0.5051 - cross_entropy: 0.8506 - val_loss: 0.7857 - val_acc: 0.3294 - val_cross_entropy: 0.7919 - lr: 1e-03
Epoch 2/40
20/20 - 10s - loss: 0.6937 - acc: 0.5216 - cross_entropy: 0.6990 - val_loss: 0.6853 - val_acc: 0.5408 - va

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4751 samples, validate on 1509 samples:
Epoch 1/2
19/19 - 9.4s - loss: 0.7893 - acc: 0.4980 - cross_entropy: 0.7956 - val_loss: 0.8829 - val_acc: 0.3234 - val_cross_entropy: 0.8901 - lr: 1e-08
Epoch 2/2
19/19 - 10s - loss: 0.7909 - acc: 0.4936 - cross_entropy: 0.7973 - val_loss: 0.8363 - val_acc: 0.3320 - val_cross_entropy: 0.8431 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4751 samples, validate on 1509 samples:
Epoch 1/40
19/19 - 10s - loss: 0.9020 - acc: 0.4963 - cross_entropy: 0.9094 - val_loss: 0.7576 - val_acc: 0.3254 - val_cross_entropy: 0.7635 - lr: 1e-03
Epoch 2/40
19/19 - 10s - loss: 0.7017 - acc: 0.5083 - cross_entropy: 0.7070 - val_loss: 0.7198 - val_acc: 0.3380 - v

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4786 samples, validate on 1491 samples:
Epoch 1/2
19/19 - 9.5s - loss: 0.7419 - acc: 0.5075 - cross_entropy: 0.7477 - val_loss: 0.6763 - val_acc: 0.5634 - val_cross_entropy: 0.6814 - lr: 1e-08
Epoch 2/2
19/19 - 9.5s - loss: 0.7443 - acc: 0.4946 - cross_entropy: 0.7501 - val_loss: 0.6834 - val_acc: 0.5386 - val_cross_entropy: 0.6886 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 4786 samples, validate on 1491 samples:
Epoch 1/40
19/19 - 9.4s - loss: 0.7682 - acc: 0.5146 - cross_entropy: 0.7742 - val_loss: 0.7525 - val_acc: 0.3327 - val_cross_entropy: 0.7584 - lr: 1e-03
Epoch 2/40
19/19 - 10s - loss: 0.6923 - acc: 0.5176 - cross_entropy: 0.6976 - val_loss: 0.6809 - val_acc: 0.5580 -

In [ ]:

Out=pd.DataFrame()
Or_model,no_model,model= BuildModel()
no_model.reset_Sample_size(1)
for j in range(20):
  for i in range(5):
    model.load_weights('ShortModel{}'.format(i))
    Out[i+j*5]=torch.softmax(torch.tensor(model.predict(Xout.fillna(0).values,batch_size=256)),dim=-1)[:,1]

[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto c

In [ ]:
Out.to_csv('OutLocal.csv')

In [ ]:
  X_train ,y_train ,X_test,y_test,Itest= datasplit(X,Train,size=100000,PP=[0.75,0.25])
  X_train=X_train[(y_train<3) | ( (y_train>7) &  (y_train<13))  ].fillna(0)
  y_train=y_train[(y_train<3) | ( (y_train>7) &  (y_train<13))  ]
  Ytruth = y_test*1.0
  Xtruth =  X_test.fillna(0)*1.0
  X_test=X_test[(y_test.Y<3) | ( (y_test.Y>7) &  (y_test.Y<13))  ].fillna(0)
  y_test=y_test[(y_test.Y<3) | ( (y_test.Y>7) &  (y_test.Y<13))  ]

  Ytruth = y_test*1.0
  y_test=pd.DataFrame(np.where(y_test>3,1,0).astype(int)).fillna(0)
  y_train=pd.DataFrame(np.where(y_train>3,1,0).astype(int)).fillna(0)

  model2=TrainMolde('ShortModel2',X_train,y_train,X_test,y_test)

NameError: name 'TrainMolde' is not defined

In [ ]:
y_test.hist(bins=100)

In [ ]:






TrainMolde('ShortModel1',X_train,y_train,X_test,y_test)
TrainMolde('ShortModel2',X_train,y_train,X_test,y_test)

In [ ]:
model.load_weights(checkpoint)


In [ ]:
df=pd.DataFrame()
df['L']=y_test[0].values
df['P'] = torch.softmax(torch.tensor(model.predict(X_test.fillna(0).values)),dim=-1)[:,1].numpy()

In [ ]:
plt.scatter(df['P'],df['L'],s=0.1)

In [ ]:
plt.plot(df.sort_values('P').rolling(100).mean().P.values[::100],df.sort_values('P').rolling(100).mean().L.values[::100])

In [ ]:
def calculate_confusion_matrix(df):
    # Calculate the confusion matrix from the DataFrame
    grouped = df.groupby(['L', 'P']).size().unstack(fill_value=0)
    return grouped.to_numpy()

In [ ]:
pd.DataFrame(calculate_confusion_matrix(df.round()))

In [ ]:
class Inference( ):
  def __init__(self,M):
    self.M=M
  def predict(self,X):
   Out=1
   for model in self.M:
       Out=Out*model.predict(X)
   return np.power(Out,1/len(self.M))

In [ ]:
y_test.min()

In [ ]:
N=2
M_G=[None]*N
scaler_x=[None]*N
scaler_y=[None]*N

for i in range(N):
  X_train ,y_train ,X_test,y_test,Itest= datasplit(X,Train,size=50000,PP=[0.75,0.25])
  X_train=X_train[(y_train<11)]# | (y_train>37.5)  ]
  y_train=y_train[(y_train<11)]# | (y_train>37.5)  ]
  X_test=X_test[(y_test.Y<11)]# | (y_test.Y>37.5)  ]
  y_test=y_test[(y_test.Y<11)]# | (y_test.Y>37.5)  ]
 # y_test=pd.DataFrame(np.where(y_test>1,0,1).astype(int))
 # y_train=pd.DataFrame(np.where(y_train>1,0,1).astype(int))
  M_G[i]=simple_lgb_regression(X_train,y_train,X_test,y_test)

ModelG = Inference(M_G )


In [ ]:

N=4
M_G=[None]*N
scaler_x=[None]*N
scaler_y=[None]*N


X_train ,y_train ,X_test,y_test,Itest= datasplit(X,Train,C_ID,size=120400,PP=[0.7,0.3])
X_train=X_train[(y_train<19) | (y_train>40.5)  ]
y_train=y_train[(y_train<19) | (y_train>40.5)  ]
X_test=X_test[(y_test.Y<19) | (y_test.Y>40.5)  ]
y_test=y_test[(y_test.Y<19) | (y_test.Y>40.5)  ]
ModelG2=simple_lgb_regression(X_train,y_train,X_test,y_test)

In [ ]:
y_train.hist()

In [ ]:
X_train ,y_train ,X_test,y_test,Itest= datasplit(X,Train,C_ID,size=100400,PP=[0.7,0.3])

N=4
M_L=[None]*N
scaler_x_l=[None]*N
scaler_y_l=[None]*N


X_train=X_train[(y_train<10)  ]
y_train=y_train[(y_train<10)   ]
X_test=X_test[(y_test.Y<19)   ]
y_test=y_test[(y_test.Y<19)   ]


ModelL=simple_lgb_regression(X_train,y_train,X_test,y_test)



In [ ]:
df=pd.DataFrame()
df['pred']=ModelG.predict(X_test).round(1)
df['labe']=y_test.values
df=df.sort_values('pred')
plt.scatter(df.pred.values,df.labe.values,s=1)

In [ ]:
plt.plot(df.groupby('pred').mean())
plt.plot(df.groupby('pred').quantile(0.9))
plt.plot(df.groupby('pred').quantile(0.1))

In [ ]:
df

In [ ]:
df[np.round(df.pred) != df.labe ].mean()

In [ ]:
1-len(df[np.round(df.pred) != df.labe ]) /len(df)

In [ ]:
plt.scatter(PredOut.Gscore,PredOut.G2score,s=0.11)

In [ ]:
plt.scatter(PredOut.G2score,PredOut.Gscore)
plt.plot([0,1],[0.5,0.5])
plt.plot([0.5,0.5],[0,1])

In [ ]:
#ModelL = Inference(M_L ,scaler_x_l, scaler_y_l)
#ModelG = Inference(M_G ,scaler_x, scaler_y)
PredOut=pd.DataFrame()
PredOut['Lscore']=ModelG.predict(Xout)
PredOut['Gscore']=ModelG.predict(Xout)
PredOut['G2score']=ModelG.predict(Xout)


PredOut['gen']=Out.gen
PredOut['ChassisId']=Out.ChassisId_encoded
PredOut['Timesteps']=Out.Timesteps
PredOut['MinLscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').min().Lscore)
PredOut['MinGscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').min().Gscore)
PredOut['MeanGscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').Gscore.mean())

PredOut['MaxLscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').max().Lscore)
PredOut['MaxGscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').max().Gscore)

PredOut['DGscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').first().Gscore) - PredOut.ChassisId.map( PredOut.groupby('ChassisId').last().Gscore)
PredOut['DLscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').first().Lscore) - PredOut.ChassisId.map( PredOut.groupby('ChassisId').last().Lscore)

PredOut['O']=PredOut.Timesteps-  PredOut.ChassisId.map( PredOut.groupby('ChassisId').min().Timesteps)


In [ ]:
PredOut['final']='U'

PredOut.loc[  (PredOut.gen=='gen1') & (PredOut.Lscore<=10), 'final']='High'
PredOut.loc[(PredOut.gen=='gen1') & (PredOut.Lscore>10), 'final']='Medium'

PredOut.loc[  (PredOut.gen=='gen2') & (PredOut.Lscore<=10), 'final']='High'
PredOut.loc[(PredOut.gen=='gen2') & (PredOut.Lscore>10), 'final']='Medium'

In [ ]:
Out.f__235.diff()

In [ ]:


PredOut.loc[PredOut.O==0, 'final']='Medium'
PredOut.loc[PredOut.O==1, 'final']='Medium'
PredOut.loc[PredOut.O==2, 'final']='Medium'
PredOut.loc[PredOut.O==3, 'final']='Medium'
#PredOut.loc[ (PredOut.O==3) & (PredOut.MinGscore<3), 'final']='High'



PredOut.loc[PredOut.O==4, 'final']='High'
#PredOut.loc[ (PredOut.O==4) & (PredOut.MinGscore>10), 'final']='Medium'


#PredOut.loc[ (PredOut.O==4) & (PredOut.MinGscore<3), 'final']='High'




PredOut.loc[PredOut.O==5, 'final']='High'

PredOut.loc[PredOut.O==6, 'final']='High'

PredOut.loc[PredOut.O==7, 'final']='High'

PredOut.loc[PredOut.O==8, 'final']='High'
PredOut.loc[PredOut.O==9, 'final']='High'


PredOut.loc[ (PredOut.gen=='gen2') & (PredOut.MeanGscore<0.5), 'final']='Low'
PredOut.loc[(PredOut.gen=='gen1') & (PredOut.MeanGscore<0.5), 'final']='Low'

In [ ]:
PredOut.to_csv('Local.csv')

In [ ]:
#Z=pd.DataFrame(Out[PredOut.final!='Low'].af1__4.values.reshape((-1,10)))
#Z.index=Out[PredOut.final!='Low'].index[::10]
#i=Z[Z.diff(axis=1)[8].abs()>0].index+2
#PredOut.loc[i,'final']='High'
#PredOut.loc[i+1,'final']='High'

In [ ]:
PredOut[PredOut.final!='251']

In [ ]:
Z=pd.DataFrame(PredOut[PredOut.final!='251'].Lscore.values.reshape((-1,10)))
Z[Z[9]<3]


In [ ]:
Z=pd.DataFrame(PredOut.Gscore.values.reshape((3359,10 )))
from scipy.stats import gmean

Z[ (Z.max(axis=1)>0.5)  &  (Z.min(axis=1)<0.5)].apply(gmean,axis=1)

In [ ]:
PredOut[PredOut.final=='Medium'].MinGscore.hist(bins=100)

In [ ]:

# Extract feature importance
importance = ModelG.M[0].feature_importance()
feature_names = ModelG.M[0].feature_name()

# Create a DataFrame for visualization
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importance})
#feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)


# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['feature'], feature_importance_df['importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
feature_importance_df[feature_importance_df.importance<500]
#feature_importance_df[feature_importance_df.feature=='f__234']

In [ ]:
Gen1 = PredOut[PredOut['gen']=='gen1']
Gen2 = PredOut[PredOut['gen']=='gen2']



In [ ]:
Gen2#.G2score.hist(bins=100)

In [ ]:
Gen1.MinLscore.hist(bins=100)

In [ ]:
Gen2.Lscore.hist(bins=100)

In [ ]:
#Gen1[Gen1.MinGscore<200].Lscore.hist(bins=100)
Gen2.Gscore.hist(bins=100)

In [ ]:
#Gen1[Gen1.MinGscore<200].Lscore.hist(bins=100)
Gen2[Gen2.MinGscore<30].Lscore.hist(bins=100)

In [ ]:
Gen1[Gen1.MinGscore<20].Lscore.hist(bins=100)

In [ ]:
Gen2[Gen2.MinGscore<30].Lscore.mean()


In [ ]:
Gen1[(Gen1.MinGscore<20)&(Gen1['O']==7)].Lscore.hist()

In [ ]:
Gen1[Gen1.MinGscore<20].Lscore.mean()


In [ ]:
len( Gen2[ ( Gen2.MinGscore<200) &  ( Gen2.Lscore<=14)] ) / len( Gen2[ ( Gen2.MinGscore<200) &  ( Gen2.Lscore<=25)] )

In [ ]:
len( Gen1[ ( Gen1.MinGscore<200) &  ( Gen1.Lscore<=10)] ) / len( Gen1[ ( Gen1.MinGscore<200) &  ( Gen1.Lscore<=25)] )

In [ ]:
plt.scatter(Gen2.MaxLscore,Gen2.MinLscore,s=1)

In [ ]:
Variants['pred']='U'
Variants.loc[Variants.index.isin(PredOut.ChassisId.unique()), 'pred']=PredOut.groupby('ChassisId').last().final.loc[Variants.loc[Variants.index.isin(PredOut.ChassisId.unique()), 'pred'].index]

In [ ]:
Variants.groupby(['v__11','pred']).count()

In [ ]:
PredOut['final']='U'
PredOut['gf']='U'
PredOut['HMs']=1






PredOut.loc[PredOut.MaxGscore>35, 'final']='Low'
PredOut.loc[PredOut.MinGscore>25, 'final']='Low'
PredOut.loc[PredOut.MaxGscore>35, 'gf']='Low'
PredOut.loc[PredOut.MinGscore>25, 'gf']='Low'


PredOut.loc[PredOut.MaxGscore<21, 'final']='HM'
PredOut.loc[PredOut.MinGscore<12, 'final']='HM'
PredOut.loc[PredOut.MaxGscore<21, 'gf']='HM'
PredOut.loc[PredOut.MinGscore<12, 'gf']='HM'


PredOut.loc[ ( PredOut['final']=='U') &(PredOut.MinLscore<8),'gf' ]='HM'

PredOut.loc[ ( PredOut['final']=='U') &(PredOut.MinLscore<8),'final' ]='HM'


PredOut.loc[ ( PredOut['final']=='U') , 'gf']='Low'
PredOut.loc[ ( PredOut['final']=='U') , 'final']='Low'


In [ ]:

PredOut.loc[(PredOut['gf']=='HM' ) & (PredOut['Lscore']<=9),'final']= 'High'
PredOut.loc[(PredOut['gf']=='HM' ) & (PredOut['Lscore']>9),'final']= 'Medium'


PredOut.loc[(PredOut['gf']=='HM' )    & ( PredOut['MinLscore']<3 ),'final']= 'High'
PredOut.loc[(PredOut['gf']=='HM' )    & ( PredOut['MinLscore']>10 ),'final']= 'Medium'




PredOut.loc[(PredOut['gf']=='HM' ) & (PredOut['O']==9),'final']= 'High'
PredOut.loc[(PredOut['gf']=='HM' ) & (PredOut['O']==0),'final']= 'Medium'

In [ ]:
PredOut[(PredOut['final']=='U' )]

In [ ]:
PredOut.groupby('final').Lscore.count()

In [ ]:
PredOut[PredOut.final=='Medium'].O.hist(bins=111)

In [ ]:
plt.scatter( Gen1[Gen1.Gscore<20].Gscore ,Gen1[Gen1.Gscore<20].Lscore)

In [ ]:
PredOut[PredOut.Gscore<20].Lscore.hist(bins=100)

In [ ]:
Model = Inference([model1], scaler_x, scaler_y)

df=pd.DataFrame()
df['Y']=y_test
df['pred']=Model.predict(X_test)
df['acc']=0

df['P']=1
df.loc[df.pred<9,'P']=0
#df.loc[df.pred<10,'P']=2
df['L']=1

#df.loc[df.Y<18,'L']=1
df.loc[df.Y<9,'L']=0
df.loc[df.L==df.P,'acc']=1
calculate_confusion_matrix(df)

In [ ]:
OutX

In [ ]:
a=df.sort_values('pred').rolling(20 ).mean()
plt.plot(a.pred,a.acc)

In [ ]:
Model = Inference([model5], scaler_x, scaler_y)


In [ ]:


#Model = Inference([model2], scaler_x, scaler_y)

df=pd.DataFrame()
df['Y']=y_test
df['pred']=Model.predict(X_test)
df['acc']=0
df.loc[ (df.Y>30) & (df.pred>30) ,'acc']=1
df.loc[ (df.Y<=30) & (df.pred<=30) ,'acc']=1
df['P']=0
df.loc[df.pred<30,'P']=1
#df.loc[df.pred<10,'P']=2
df['L']=0
#df.loc[df.Y<18,'L']=1


df.loc[df.Y<30,'L']=1
#df['ID']=Train.loc[Itest].ChassisId_encoded.map(C_ID).values

In [ ]:
a=df.sort_values('pred')
a['r']=range(len(a))
ma=a.rolling(10).max()
mi=a.rolling(10).min()
s=a.rolling(10).std()

a=a.rolling(10).mean()

plt.plot(a.pred,a.Y)
plt.plot(a.Y,a.Y)
plt.plot(a.pred,a.Y+s.Y)
plt.plot(a.pred,a.Y-s.Y)

#plt.plot(a.pred,mi.Y)
plt.plot([9,9],[0.5,1])
plt.plot([18,18],[0.5,1])

In [ ]:
plt.plot(a.Y,a.acc)
#plt.plot(a.Y,a.Y)

In [ ]:
def f(Y):
  return np.where(Y<19,0,1)

In [ ]:
A=pd.DataFrame()
A['maxpred']=df.groupby('ID').max().pred
A['minpred']=df.groupby('ID').min().pred
df['classs']=df.Y.map(f)
df['Pclasss']=df.pred.map(f)
A['pred']=df.groupby('ID').mean().Pclasss

A['Y']=df.groupby('ID').mean().classs
A['C']=df.groupby('ID').count().Y
A

In [ ]:
df.groupby('ID').max()

In [ ]:
a[a.Y<11].mean()

In [ ]:
# Sample data
# Assuming X_train, y_train, X_test, y_test are already defined
# X_train, X_test: 2D numpy arrays of shape (num_samples, num_features)
# y_train, y_test: 1D numpy arrays of shape (num_samples,)

# Example usage




# Predict and calculate metrics
model.eval()
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).cuda()
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).cuda()
    y_pred_tensor = model(X_test_tensor)
    y_pred = y_pred_tensor.cpu().numpy().flatten()

    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    test_f1 = macro_f1(y_test, y_pred)

    print("\nFinal Evaluation on Test Set:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print(f"Macro F1 Score: {test_f1:.4f}")


In [ ]:
df['P']=0
df.loc[df.pred<28,'P']=1
#df.loc[df.pred<10,'P']=2
df['L']=0
#df.loc[df.Y<18,'L']=1
df.loc[df.Y<28,'L']=1
df
print(calculate_confusion_matrix(df))
calculate_macro_f1(calculate_confusion_matrix(df))


In [ ]:
L=[]
for i in range(20,35):
  df['P']=0
  df.loc[df.pred<i,'P']=1
  #df.loc[df.pred<10,'P']=2
  df['L']=0
  #df.loc[df.Y<18,'L']=1
  df.loc[df.Y<i,'L']=1
  df
  L.append( calculate_macro_f1(calculate_confusion_matrix(df)) )

In [ ]:
plt.plot(range(20,35),L)

In [ ]:
X_train.iloc[0].sort_values()

In [ ]:
def calculate_confusion_matrix(df):
    # Calculate the confusion matrix from the DataFrame
    grouped = df.groupby(['L', 'P']).size().unstack(fill_value=0)
    return grouped.to_numpy()

def calculate_macro_f1(confusion_matrix):
    # Initialize variables to store precision, recall, and F1 scores for each class
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Iterate through each class (assume square confusion matrix)
    for i in range(len(confusion_matrix)):
        TP = confusion_matrix[i, i]
        FP = np.sum(confusion_matrix[:, i]) - TP
        FN = np.sum(confusion_matrix[i, :]) - TP

        if TP + FP == 0:
            precision = 0
        else:
            precision = TP / (TP + FP)

        if TP + FN == 0:
            recall = 0
        else:
            recall = TP / (TP + FN)

        if precision + recall == 0:
            f1 = 0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    # Calculate the macro F1 score
    macro_f1 = np.mean(f1_scores)
    return macro_f1

In [ ]:
#Model = Inference(model, scaler_x, scaler_y)

df=pd.DataFrame()
df['Y']=y_test
df['pred']=Model.predict(X_test)
df['ID']=Train.loc[Itest].ChassisId_encoded.map(C_ID).values

In [ ]:
Model.predict(X_test)

In [ ]:
Model.predict(X_test)

In [ ]:
X_test[0]

In [ ]:
df['P']=0
df.loc[df.pred<18,'P']=1
df.loc[df.pred<10,'P']=2
df['L']=0
df.loc[df.Y<18,'L']=1
df.loc[df.Y<9,'L']=2

In [ ]:
df

In [ ]:

df['P']=0
df.loc[df.pred<19,'P']=1
df.loc[df.pred<10,'P']=2
df['L']=0
df.loc[df.Y<19,'L']=1
df.loc[df.Y<10,'L']=2


calculate_macro_f1(calculate_confusion_matrix(df))
#calculate_confusion_matrix(df)

In [ ]:
len(df[np.sign(df.P)==np.sign(df.L)])/len(df)

In [ ]:
df['P']=0
df.loc[df.pred<19,'P']=1
df.loc[df.pred<10,'P']=2
df['L']=0
df.loc[df.Y<19,'L']=1
df.loc[df.Y<10,'L']=2


calculate_macro_f1(calculate_confusion_matrix(df[df.ID==0.0]))

In [ ]:
calculate_confusion_matrix(df)

In [ ]:
np.corrcoef(df.pred.values,df.Y.values)

In [ ]:
pd.DataFrame([[15281,     6,    60] ,[ 5 ,2076]])

In [ ]:
PredOut=pd.DataFrame()
PredOut['score']=Model.predict(Xout)
PredOut['gen']=Out.gen

In [ ]:
PredOut['r']=PredOut.sort_values('score').index / len(PredOut)

In [ ]:
PredOut.loc[PredOut.gen=='gen2','r']=PredOut.sort_values('score').loc[PredOut.sort_values('score').gen=='gen2','r'].reset_index().index / len(PredOut.loc[PredOut.gen=='gen2','r'])

In [ ]:
PredOut['pred']='Low'
#PredOut.loc[(PredOut.r<0.045) & (PredOut.gen=='gen1'),'pred']='High'

In [ ]:
PredOut.groupby(['gen','pred']).count()

In [ ]:
PredOut['pred']='Low'
PredOut.loc[(PredOut.score<19) & (PredOut.gen=='gen2'),'pred']='Medium'
PredOut.loc[(PredOut.score<10) & (PredOut.gen=='gen2'),'pred']='High'

PredOut.loc[(PredOut.score<19) & (PredOut.gen=='gen1'),'pred']='Medium'
PredOut.loc[(PredOut.score<10) & (PredOut.gen=='gen1'),'pred']='High'
PredOut.loc[ (PredOut.gen=='gen2'),'pred']='Medium'


In [ ]:
Z[ ( Z[5]=='Medium' )  & ( Z[4]=='High' ) ]

In [ ]:
PredOut['id']=Out.ChassisId_encoded
PredOut['time']=Out.Tss


In [ ]:
Z=pd.DataFrame()
for i in range(10):
  Z[i]=PredOut[(PredOut['time']==i) & (PredOut['gen']=='gen2') ].pred.values

In [ ]:
len(df[df.P==df.L])/len(df)

In [ ]:
Z=pd.DataFrame()
for i in range(10):
  Z[i]=PredOut[(PredOut['time']==i) & (PredOut['gen']=='gen2') ].score.values

In [ ]:
Z[(Z.max(axis=1)>19) & ((Z.mean(axis=1)<19))]

In [ ]:
((Z[0]-Z[9])).hist(bins=100)

In [ ]:
((Z[0]-Z[9])).hist(bins=100)

In [ ]:
PredOut.loc[(PredOut['pred']=='Medium') & (PredOut['time']>=7),'pred' ]='High'
PredOut.loc[(PredOut['pred']=='High') & (PredOut['time']<=2),'pred' ]='Medium'

In [ ]:
PredOut.groupby(['gen','pred']).count()

In [ ]:
(PredOut.groupby('id').score.max() -PredOut.groupby('id').score.min()).hist(bins=100)

In [ ]:
(PredOut[PredOut.gen=='gen2'].groupby('id').score.max()).hist(bins=100)

In [ ]:
plt.scatter  (     PredOut[PredOut.gen=='gen1'].groupby('id').score.min().values     , PredOut[PredOut.gen=='gen1'].groupby('id').score.max().values ,s=1              )

In [ ]:
plt.scatter  (     PredOut[PredOut.gen=='gen2'].groupby('id').score.mean().values     , PredOut[PredOut.gen=='gen2'].groupby('id').score.max().values              ,s=1 )

In [ ]:
plt.scatter  (     PredOut[PredOut.gen=='gen2'].groupby('id').score.mean().values     , PredOut[PredOut.gen=='gen2'].groupby('id').score.max().values              ,s=1 )

In [ ]:
G1 = PredOut[PredOut.gen=='gen1'].score
G2 = PredOut[PredOut.gen=='gen2'].score

In [ ]:
len(G1[G1<10])/len(G1)

In [ ]:
len(G2[G2<20])/len(G2)

In [ ]:
Train.groupby('ChassisId_encoded').last().risk_level.unique()

In [ ]:
Y=pd.read_csv('public_X_test.csv')

In [ ]:
Y[Y.ChassisId_encoded.isin(Z.ChassisId_encoded.unique())]

In [ ]:
len(X.ChassisId_encoded.unique())

In [ ]:
Y[Y.gen=='gen1' & Y.ChassisId_encoded.isin(Y[Y.gen=='gen1'].ChassisId_encoded.unique())  ]

In [ ]:
plt.scatter(Z.v__0.values,Z.v__1.values)

In [ ]:
plt.scatter(Z.v__0.values,Z.v__11.values)

In [ ]:
Y.groupby('ChassisId_encoded').last().Timesteps.hist()

In [ ]:
X.groupby('ChassisId_encoded').last().Timesteps.hist()

In [ ]:
PredOut['pred']='Medium'
#PredOut.loc[PredOut.gen=='gen1', 'pred']=PredOut.final
PredOut.loc[PredOut.gen=='gen2', 'pred']=PredOut.final

In [ ]:
#PredOut['pred']=np.nan


In [ ]:
PredOut

In [ ]:

Final=pd.read_csv('prediction_sample.csv')
Final.pred=PredOut.pred

In [ ]:
Final.to_csv('prediction.csv')

In [ ]:
PredOut
Z=pd.DataFrame()
for i in range(10):
  Z[i]=PredOut[PredOut['O']==i].pred.map({'Low':0,'Medium':1,'High':2}).values
Z.diff(axis=1).min(axis=1).hist(bins=13)

In [ ]:
PredOut[PredOut['O']==i].pred

In [ ]:
Out['pred']=PredOut.pred.values
Out.groupby('ChassisId_encoded').first()#.Timesteps.hist(bins=100)

In [ ]:
Variants['gen']= Variants.index.map(Gens.gen)

In [ ]:
Variants.index.map(Gens.gen)

In [ ]:
Variants.groupby('gen').mean()

In [ ]:
Gens=pd.concat( [ Out[['ChassisId_encoded','gen']].set_index('ChassisId_encoded'),Train[['ChassisId_encoded','gen']].set_index('ChassisId_encoded')])

In [ ]:
Gens=Gens.groupby('ChassisId_encoded').last()

In [ ]:
Variants.groupby('gen').max()

In [ ]:
M=1
Variants['ID']=0
for c in Variants.columns[:-2]:
  print(Variants[c].max())
  M=M* ( Variants[c].max()+1)
  Variants['ID']= Variants['ID']+ Variants[c]*M

In [ ]:
Variants.groupby(['ID','gen']).count().sort_values('v__0')[::-1][:10]

In [ ]:
y_true=np.random.choice([0,1,2],p=[0.91,0.09],size=10000)
L=[]
for p in np.linspace(0.8,1.0,201):
  y_pred=np.random.choice([0,1],p=[p,1-p],size=10000)

  L.append([p,   f1_score(y_true, y_pred, average='macro')])


In [ ]:
L=[]
for p1 in np.linspace(0,0.2,21):
  for p2 in np.linspace(0,0.2,21):
    y_pred=np.random.choice([0,1,2],p=[1-p1-p2,p1,p2],size=100000)
    L.append([p1,p2,   f1_score(y_true, y_pred, average='macro')])

In [ ]:
L=pd.DataFrame(L)

In [ ]:
L[L[2]==L[2].max()]

In [ ]:
Out.groupby('ChassisId_encoded')#.count().groupby('Timesteps').count()

In [ ]:
Out[Out.ChassisId_encoded==102561.0]

In [ ]:
 Train['NT']=Train.Timesteps-Train.ChassisId_encoded.map(Train.groupby('ChassisId_encoded').max().Timesteps)

In [ ]:
Train.groupby('risk_level').count().gen / len(Train)

In [ ]:
Train[Train.NT>-18].groupby('risk_level').count().gen / len(Train[Train.NT>-18])

In [ ]:
from sklearn.metrics import f1_score
p1=0.05
p2=0.07
y_true=np.random.choice([0,1,2],p=[0.88,0.05,0.07],size=100000)
y_pred=np.random.choice([0,1,2],p=[1-p1-p2,p1,p2],size=100000)
f1_score(y_true, y_pred, average='macro')

In [ ]:
DG = Train.groupby('ChassisId_encoded').first().drop(['Timesteps'	,'Tss',	'gen',	'risk_level','Y'	,'sieries_size'	,'sieries_end'	],axis=1) - Train.groupby('ChassisId_encoded').last() .drop(['Timesteps'	,'Tss',	'gen',	'risk_level','Timesteps'	,'Tss',	'gen',	'risk_level','Y'	,'sieries_size'	,'sieries_end'	],axis=1)
DG=DG[Train.groupby('ChassisId_encoded').risk_level.count()>7]

In [ ]:
Labels=Train.groupby('ChassisId_encoded').risk_level.last()
Labels  =Labels[Train.groupby('ChassisId_encoded').risk_level.count()>7]

In [ ]:
DG.f__234.hist(bins=100)

In [ ]:
DG[Labels=='Low'].f__236.hist(bins=100)

In [ ]:
DG[Labels=='High'].f__236.hist(bins=100)

In [ ]:
DG